In [1]:
"""

Goal: Create global instance IDs for masks for each video in our dataset to be able to
      train recurrent models that will need masks for the same instance across frames to
      have the same instance ID.
      
Output: Table in the form of a CSV file that contains the following rows:
        video_id, frame_number, image_id, local_instance_id, global_instance_id
        
        video_id, frame_prev_id, frame_next_id, frame_prev_instance_id, frame_next_instance_id
        
        frame_prev, frame_curr, 33001, 33003
        
        
        mapping from frame_prev_id (33001) to frame_next_id (33003)
        
Method:
  1. Use IOU to match masks between consecutive frames. This method will fail if instances
     move significantly between frames, or there are different levels/locations of occlusion
     for the masks.

"""

print("CODE")

CODE


In [2]:
# Iterate through each video

import glob

VIDEO_LIST_DIR = "../data/wad/video_list/"
IMAGE_MASK_DIR = "../data/wad/train_label/"

list_of_videos = glob.glob(VIDEO_LIST_DIR + "*.txt")
list_of_images = glob.glob(IMAGE_MASK_DIR + "*.png")

print("\nNum videos: ", len(list_of_videos), "\n\nSample from list: ", list_of_videos[:5])
print("\nNum images: ", len(list_of_images), "\n\nSample from list: ", list_of_images[:5])


Num videos:  42 

Sample from list:  ['../data/wad/video_list/road01_cam_5_video_4_image_list_train.txt', '../data/wad/video_list/road03_cam_6_video_5_image_list_train.txt', '../data/wad/video_list/road03_cam_5_video_17_image_list_train.txt', '../data/wad/video_list/road03_cam_6_video_3_image_list_train.txt', '../data/wad/video_list/road01_cam_5_video_14_image_list_train.txt']

Num images:  39222 

Sample from list:  ['../data/wad/train_label/170908_065045292_Camera_6_instanceIds.png', '../data/wad/train_label/171206_030513260_Camera_6_instanceIds.png', '../data/wad/train_label/171206_032604161_Camera_6_instanceIds.png', '../data/wad/train_label/170908_062116386_Camera_6_instanceIds.png', '../data/wad/train_label/170908_073232068_Camera_6_instanceIds.png']


In [3]:
# Helper functions

import skimage.io as io
import numpy as np
import pandas as pd
import re

import matplotlib.pyplot as plt

%matplotlib inline


def plot_consecutive_frames(frame_prev, frame_curr):
    
    frame_prev = io.imread(frame_prev)
    frame_curr = io.imread(frame_curr)
    
    fig, axarr = plt.subplots(1,2)
    fig.set_size_inches(20, 10)
    axarr[0].imshow(frame_prev)
    axarr[1].imshow(frame_curr)
    
def plot_two_masks(mask_prev, mask_curr):
    fig, axarr = plt.subplots(1,2)
    fig.set_size_inches(20, 10)
    axarr[0].imshow(mask_prev)
    axarr[1].imshow(mask_curr)

def computeIOU(prev_mask, curr_mask):
    
    # flatten masks into vectors: because frames are same dimension,
    # so insersections in the vector will match those of the matrix
    # representation of the mask, but processing the vector
    # representation will be much faster.
    
    prev_mask = prev_mask.flatten()
    curr_mask = curr_mask.flatten()
    
    # counts the total num pixels in both masks
    area_prev = np.count_nonzero(prev_mask)
    area_curr = np.count_nonzero(curr_mask)
    total_area = area_prev + area_curr
    
    # if the masks aren't even close to the same size
    if area_prev / area_curr < 0.8: return 0
    
    intersection = np.sum( (prev_mask == curr_mask) & (prev_mask) &(curr_mask) )
    union = total_area - intersection
    
    return intersection / union

def match_image_ids(image_prev_id, image_curr_id, video_id, threshold=0.7):
    
    # goal: make this run per image under 10 milliseconds
    
    """
    Returns pandas dataframe, with four columns:
    frame_prev_id, frame_curr_id, prev_instance_id, curr_instance_id
    
    the first containing the image_prev instance ids, and
    the second containing the corresponding instance ids IN THE SAME ORDER for
    image_curr instances.
    """
    
    df = {
        "frame_prev": [],
        "frame_curr": [],
        "video_id": [],
        "prev_instance_id": [],
        "curr_instance_id": []
    }
    
    image_prev_ids = []
    image_curr_ids = []
    
    image_prev = io.imread(IMAGE_MASK_DIR + image_prev_id + "_instanceIds.png")
    image_curr = io.imread(IMAGE_MASK_DIR + image_curr_id + "_instanceIds.png")
    
    unique_prev_ids = np.unique(image_prev)
    unique_curr_ids = np.unique(image_curr)
    
    for prev_id in unique_prev_ids:
        
        if prev_id == 255: continue
        
        prev_instance_mask = image_prev == prev_id
        
        curr_instances_from_same_class = unique_curr_ids[(unique_curr_ids // 1000) == (prev_id // 1000)]
        
        curr_IOUs = {}
        
        for curr_id in unique_curr_ids:
            
            if curr_id == 255: continue
            
            curr_instance_mask = image_curr == curr_id
            
            IOU = computeIOU(prev_instance_mask, curr_instance_mask)
            
            # assume that if above threshold, then it is the matching id
            # so we do not have to test other ids
            if IOU >= threshold:
                curr_IOUs[curr_id] = IOU
                break
        
        # do nothing if prev_instance has no matching instance in current frame
        
        if curr_IOUs:
            match = max(curr_IOUs, key=curr_IOUs.get)
            df["frame_prev"].append(image_prev_id)
            df["frame_curr"].append(image_curr_id)
            df["prev_instance_id"].append(prev_id)
            df["curr_instance_id"].append(match)
            df["video_id"].append(video_id)
            
        #plot_two_masks(prev_instance_mask, image_curr == match)
        
    return pd.DataFrame(df)


In [4]:
%%time

dataframe = pd.DataFrame(columns=[
    "frame_prev", "frame_curr", "video_id", "prev_instance_id", "curr_instance_id"])

num_videos = 10 # only sampling from a few videos
num_per_video = 5

np.random.shuffle(list_of_videos)

for video in list_of_videos[:num_videos]:
    
    frame_count = 0
    
    with open(video) as f:
        
        frames = f.readlines()
        frame_pairs = list(zip(frames[:-1], frames[1:]))
        
        np.random.shuffle(frame_pairs)
        
        for prev, curr in frame_pairs[:num_per_video]:
            
            frame_prev_id = re.search('^.*\\\\(.*\\.png)', prev).group(1)[:-16]
            frame_curr_id = re.search('^.*\\\\(.*\\.png)', curr).group(1)[:-16]
            
            df = match_image_ids(frame_prev_id, frame_curr_id, video)
            dataframe = dataframe.append(df, ignore_index=True)
            
            print("Completed frame pair: ", frame_prev_id, frame_curr_id)
            
            frame_count = frame_count + 1
            if frame_count > num_per_video: break
    
print(dataframe.head())

Completed frame pair:  171206_034415355_Camera_5 171206_034415513_Camera_5
Completed frame pair:  171206_034435571_Camera_5 171206_034435749_Camera_5
Completed frame pair:  171206_034210094_Camera_5 171206_034210240_Camera_5
Completed frame pair:  171206_034221867_Camera_5 171206_034221993_Camera_5
Completed frame pair:  171206_034510145_Camera_5 171206_034510283_Camera_5
Completed frame pair:  170908_061628012_Camera_5 170908_061628171_Camera_5
Completed frame pair:  170908_061503242_Camera_5 170908_061503381_Camera_5
Completed frame pair:  170908_061731241_Camera_5 170908_061731380_Camera_5
Completed frame pair:  170908_061653712_Camera_5 170908_061653851_Camera_5
Completed frame pair:  170908_061731936_Camera_5 170908_061732075_Camera_5
Completed frame pair:  170908_082032318_Camera_5 170908_082032446_Camera_5
Completed frame pair:  170908_081942810_Camera_5 170908_081943041_Camera_5
Completed frame pair:  170908_082019142_Camera_5 170908_082019290_Camera_5
Completed frame pair:  17

In [5]:
dataframe.to_csv("instance_id_mapping.csv")